In [1]:
import pandas as pd
import numpy as np
import pickle

# Install necessary libraries if not already installed
!pip install gensim
!pip install scikit-learn

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

# Specify the file path
file_path = '/content/gdrive/My Drive/all_chapters_3_rows.csv'

# Read the CSV file
df02 = pd.read_csv(file_path)

Mounted at /content/gdrive


In [4]:
import pickle

# Specify the file path
stop_words_path = '/content/gdrive/My Drive/stop_words.ob'

# Load the stop words list using pickle
with open(stop_words_path, 'rb') as fp:
    stop_words = pickle.load(fp)

In [13]:
# Display the columns of the DataFrame
df02.columns

Index(['string_values', 'string_values.1', 'string_values.2',
       'string_values.3', 'string_values.4', 'string_values.5',
       'string_values.6', 'string_values.7', 'string_values.8',
       'string_values.9', 'string_values.10', 'string_values.11',
       'string_values.12', 'string_values.13', 'string_values.14',
       'string_values.15', 'string_values.16', 'Ch_No'],
      dtype='object')

In [17]:
# Declare a list that is to be converted into a column
ch_no = ['ear_nose', 'musculoskeletal', 'respiratory']

# Repeat the list elements to match the length of the DataFrame
repeated_ch_no = ch_no * (len(df02) // len(ch_no)) + ch_no[:len(df02) % len(ch_no)]

# Add a new column named 'Ch_No' and assign it the repeated values
df02['Ch_No'] = repeated_ch_no

# Display the updated DataFrame
df02.head()  # You can adjust the number of rows to display if needed

,string_values,string_values.1,string_values.2,string_values.3,string_values.4,string_values.5,string_values.6,string_values.7,string_values.8,string_values.9,string_values.10,string_values.11,string_values.12,string_values.13,string_values.14,string_values.15,string_values.16,Ch_No
0,introduction bone muscle ligament musculos sha...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ear_nose
1,nd trunk presenting jarring aggravates backach...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,musculoskeletal
2,ia muscle compress ischemia necrosis ensuing r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,respiratory
3,gatifloxacin macrolide azithromycin clarithrom...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ear_nose
4,titium destroyed replaced honeycombing cyst fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,musculoskeletal


In [18]:
df02

,string_values,string_values.1,string_values.2,string_values.3,string_values.4,string_values.5,string_values.6,string_values.7,string_values.8,string_values.9,string_values.10,string_values.11,string_values.12,string_values.13,string_values.14,string_values.15,string_values.16,Ch_No
0,introduction bone muscle ligament musculos sha...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ear_nose
1,nd trunk presenting jarring aggravates backach...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,musculoskeletal
2,ia muscle compress ischemia necrosis ensuing r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,respiratory
3,gatifloxacin macrolide azithromycin clarithrom...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ear_nose
4,titium destroyed replaced honeycombing cyst fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,musculoskeletal
5,rrelated hydrops h endolymph build obstruction...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,respiratory
6,ge vascularization whichever method tumor must...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ear_nose


In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer with stop words
count_vectorizer = CountVectorizer(stop_words=stop_words)

# Fit and transform the 'string_values' column using the CountVectorizer
doc_word_cv = count_vectorizer.fit_transform(df02['string_values'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['14', 'based', 'children', 'db', 'ground', 'information', 'inspiration', 'listlessness', 'seal', 'segment'] not in stop_words.
  warnings.warn(


In [20]:
import pandas as pd

# Create a DataFrame from the sparse matrix
df_doc_word_cv = pd.DataFrame(doc_word_cv.toarray(), index=df02['Ch_No'], columns=count_vectorizer.get_feature_names_out())

# Display the first few rows of the DataFrame
df_doc_word_cv.head()


,aap,abdomen,abdu,abduct,abducted,abducting,abduction,abductor,abgs,ability,...,ysis,ysitis,zealand,zed,zheng,zinc,zone,zoster,zygote,µm
Ch_No,,,,,,,,,,,,,,,,,,,,,
ear_nose,0,0,0,2,2,2,3,1,0,1,...,0,0,0,0,0,0,0,0,0,0
musculoskeletal,0,1,1,0,0,0,7,0,0,1,...,1,1,1,0,0,0,0,0,1,0
respiratory,0,1,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
ear_nose,0,0,0,0,0,0,0,0,0,3,...,0,0,1,1,0,0,2,0,0,0
musculoskeletal,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,1,4,0,0,5


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer with stop words
tfidf = TfidfVectorizer(stop_words=stop_words)

# Fit and transform the 'string_values' column using the TfidfVectorizer
doc_word_tfidf = tfidf.fit_transform(df02['string_values'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['14', 'based', 'children', 'db', 'ground', 'information', 'inspiration', 'listlessness', 'seal', 'segment'] not in stop_words.
  warnings.warn(


In [22]:
import pandas as pd

# Create a DataFrame from the sparse matrix
df_doc_word_tfidf = pd.DataFrame(doc_word_tfidf.toarray(), index=df02['Ch_No'], columns=tfidf.get_feature_names_out())

# Display the first few rows of the DataFrame
df_doc_word_tfidf.head()

,aap,abdomen,abdu,abduct,abducted,abducting,abduction,abductor,abgs,ability,...,ysis,ysitis,zealand,zed,zheng,zinc,zone,zoster,zygote,µm
Ch_No,,,,,,,,,,,,,,,,,,,,,
ear_nose,0.000000,0.000000,0.000000,0.016346,0.013569,0.016346,0.020353,0.008173,0.000000,0.003882,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
musculoskeletal,0.000000,0.006644,0.008003,0.000000,0.000000,0.000000,0.046505,0.000000,0.000000,0.003802,...,0.006644,0.008003,0.006644,0.000000,0.000000,0.000000,0.000000,0.0,0.008003,0.00000
respiratory,0.000000,0.008605,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010366,0.009848,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
ear_nose,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016055,...,0.000000,0.000000,0.009352,0.009352,0.000000,0.000000,0.018704,0.0,0.000000,0.00000
musculoskeletal,0.006548,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003110,...,0.000000,0.000000,0.000000,0.000000,0.006548,0.006548,0.021742,0.0,0.000000,0.03274


In [23]:
from gensim import matutils

# Convert sparse matrix of counts to a Gensim corpus
corpus = matutils.Sparse2Corpus(doc_word_cv)

In [24]:
# Create a dictionary mapping from integer ids to words
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [25]:
from gensim import models

# Create LDA model
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=5)

In [28]:
# Print the topics generated by the LDA model
lda.print_topics(num_topics=3)

[(0,
  '0.278*"abducted" + 0.271*"abdu" + 0.263*"abduct" + 0.040*"abducting" + 0.004*"aap" + 0.004*"abduction" + 0.002*"abdomen" + 0.000*"penicillin" + 0.000*"penetration" + 0.000*"penetrating"'),
 (1,
  '0.301*"abdomen" + 0.284*"aap" + 0.252*"abducting" + 0.008*"abducted" + 0.005*"abduct" + 0.005*"abdu" + 0.004*"abduction" + 0.000*"penicillin" + 0.000*"penetration" + 0.000*"penetrating"'),
 (2,
  '0.000*"abducted" + 0.000*"abdomen" + 0.000*"aap" + 0.000*"abducting" + 0.000*"abdu" + 0.000*"abduct" + 0.000*"abduction" + 0.000*"penicillin" + 0.000*"penetration" + 0.000*"penetrating"')]

In [30]:
!pip install corextopic

import numpy as np
from corextopic import corextopic as ct
from corextopic import vis_topic as vt

# Define the vocabulary
words = list(count_vectorizer.vocabulary_.keys())

In [31]:
# Create a Corex topic model with 3 hidden topics
topic_model = ct.Corex(n_hidden=3, words=words, seed=1)

# Fit the topic model to the data
topic_model.fit(doc_word_cv, words=words, docs=df02['string_values'])

In [32]:
topics = topic_model.get_topics()
for n, topic in enumerate(topics):
    topic_words, _, _ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: exacerbation,injection,drive,advancing,diaphragmbecause,avulsion,colloid,punched,sepsis,hyperventilation
1: turnover,tamponade,tuck,mobi,liability,cis,dystrophy,cava,smoking,soften
2: institute,painfully,palatoplasty,dodia,elia,walk,positively,trimethoprim,land,befor


In [33]:
from sklearn.decomposition import TruncatedSVD

# Create a TruncatedSVD model with 3 components
lsa = TruncatedSVD(3)

# Fit the TruncatedSVD model to the document-word matrix
doc_topic = lsa.fit_transform(doc_word_cv)

# Print the explained variance ratio
print(lsa.explained_variance_ratio_)

[0.12768664 0.349331   0.20529573]


In [34]:
import pandas as pd

# Create a DataFrame representing the topic-word matrix
topic_word = pd.DataFrame(lsa.components_.round(3),
                          index=['component'+str(i) for i in range(3)],
                          columns=count_vectorizer.get_feature_names_out())

# Print the topic-word matrix
print(topic_word)

              aap  abdomen   abdu  abduct  abducted  abducting  abduction  \
component0  0.002    0.002  0.001   0.004     0.005      0.004      0.015   
component1  0.003   -0.002 -0.002  -0.005    -0.004     -0.005     -0.022   
component2 -0.003    0.002 -0.001  -0.003     0.000     -0.003     -0.009   

            abductor   abgs  ability  ...   ysis  ysitis  zealand    zed  \
component0     0.002  0.001    0.011  ...  0.003   0.001    0.002  0.002   
component1    -0.003  0.000   -0.000  ... -0.001  -0.002   -0.002  0.001   
component2    -0.001  0.003    0.012  ...  0.002  -0.001    0.002  0.006   

            zheng   zinc   zone  zoster  zygote     µm  
component0  0.002  0.002  0.009   0.001   0.001  0.010  
component1  0.003  0.003  0.013   0.001  -0.002  0.016  
component2 -0.003 -0.003 -0.005   0.003  -0.001 -0.013  

[3 rows x 5838 columns]


In [35]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    global tem_list
    tem_list = []
    for ix, topic in enumerate(model.components_):
        inner_tem_list = []
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '", topic_names[ix], "'")
        top_words = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        print(", ".join(top_words))
        inner_tem_list.append(", ".join(top_words))
        tem_list.append(inner_tem_list)

In [36]:
result1 = display_topics(lsa, count_vectorizer.get_feature_names_out(), 20)


Topic  0
bone, ear, muscle, otitis, hearing, membrane, airway, sur, bleeding, oxygen, hip, deformity, externa, obstruction, gery, cord, ventilation, attack, nose, swelling

Topic  1
ear, otitis, hearing, externa, membrane, silicosis, cwp, nose, dust, obstruction, sinus, sinusitis, oxygen, conduction, mastoiditis, myringitis, coal, airway, dyspnea, silica

Topic  2
airway, ventilation, obstruction, oxygen, breathing, bleeding, sinusitis, copd, nose, cord, dyspnea, septum, polyp, acidosis, mouth, artery, hg, place, inspiration, pr


In [37]:
final_dic = {
    "Bone": tem_list[0],
    "Ear": tem_list[1],
    "Breathing": tem_list[2]
}

In [44]:
final_dic

{'Bone': ['bone, ear, muscle, otitis, hearing, membrane, airway, sur, bleeding, oxygen, hip, deformity, externa, obstruction, gery, cord, ventilation, attack, nose, swelling'],
 'Ear': ['ear, otitis, hearing, externa, membrane, silicosis, cwp, nose, dust, obstruction, sinus, sinusitis, oxygen, conduction, mastoiditis, myringitis, coal, airway, dyspnea, silica'],
 'Breathing': ['airway, ventilation, obstruction, oxygen, breathing, bleeding, sinusitis, copd, nose, cord, dyspnea, septum, polyp, acidosis, mouth, artery, hg, place, inspiration, pr']}

In [45]:
import pandas as pd

# Create DataFrame from the dictionary
tem_df = pd.DataFrame.from_dict(final_dic, orient='index')

# Display the DataFrame
tem_df

,0
Bone,"bone, ear, muscle, otitis, hearing, membrane, ..."
Ear,"ear, otitis, hearing, externa, membrane, silic..."
Breathing,"airway, ventilation, obstruction, oxygen, brea..."


In [46]:
# Declare a list that is to be converted into a column
d_name = ['musculoskeletal', 'ear_nose', 'respiratory']

# Add a new column named 'D_Name' and assign it the values from the list
tem_df['D_Name'] = d_name

# Display the updated DataFrame
tem_df

,0,D_Name
Bone,"bone, ear, muscle, otitis, hearing, membrane, ...",musculoskeletal
Ear,"ear, otitis, hearing, externa, membrane, silic...",ear_nose
Breathing,"airway, ventilation, obstruction, oxygen, brea...",respiratory


In [47]:
# Declare a list that is to be converted into a column
d_name = ['musculoskeletal', 'ear_nose', 'respiratory']

# Add a new column named 'D_Name' and assign it the values from the list
tem_df['D_Name'] = d_name

# Display the updated DataFrame
tem_df

,0,D_Name
Bone,"bone, ear, muscle, otitis, hearing, membrane, ...",musculoskeletal
Ear,"ear, otitis, hearing, externa, membrane, silic...",ear_nose
Breathing,"airway, ventilation, obstruction, oxygen, brea...",respiratory


In [48]:
tem_df.columns

Index([0, 'D_Name'], dtype='object')

In [49]:
tem_df = tem_df.rename(columns={0: 'Description'})
tem_df

,Description,D_Name
Bone,"bone, ear, muscle, otitis, hearing, membrane, ...",musculoskeletal
Ear,"ear, otitis, hearing, externa, membrane, silic...",ear_nose
Breathing,"airway, ventilation, obstruction, oxygen, brea...",respiratory


In [50]:
tem_df.to_csv('diseases_with_description.csv', index=False)